## create a BQ dataset for dataset2 titled "{source}_modeled"

In [42]:
dataset_id = "WildLand_Incidents_staging"

In [19]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset
'trusty-wavelet-252622:WildLand_Incidents_staging' already exists.


## Import files from Storage into BQ staging dataset and verify they are present with a SELECT COUNT(*)

### WildLandIncidents2010

In [66]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.WildLandIncidents_2010 \
"gs://hermosillobazan_datasets/WildLand_Incidents_2010-2013/WildLandIncidents2010.csv"\
\state:STRING,fdid:STRING,inc_date:INTEGER,inc_no:STRING,exp_no:INTEGER,version:STRING,latitude:FLOAT,longitude:FLOAT,township:FLOAT,north_sou:STRING,range:INTEGER,east_west:STRING,section:INTEGER,subsection:STRING,meridian:STRING,area_type:STRING,fire_cause:STRING,hum_fact1:STRING,hum_fact2:STRING,hum_fact3:STRING,hum_fact4:STRING,hum_fact5:STRING,hum_fact6:STRING,hum_fact7:STRING,hum_fact8:STRING,fact_ign1:STRING,fact_ign2:STRING,supp_fact1:STRING,supp_fact2:STRING,supp_fact3:,heat_sourc:STRING,mob_prop:STRING,eq_inv_ign:STRING,nfdrs_id:STRING,weath_type:STRING,wind_dir:STRING,wind_speed:INTEGER,air_temp:INTEGER,rel_humid:INTEGER,fuel_moist:INTEGER,dangr_rate:STRING,bldg_inv:INTEGER,bldg_thr:INTEGER,acres_burn:FLOAT,crop_burn1:STRING,crop_burn2:STRING,crop_burn3:STRING,undet_burn:INTEGER,tax_burn:INTEGER,notax_burn:INTEGER,local_burn:INTEGER,couty_burn:INTEGER,st_burn:INTEGER,fed_burn:INTEGER,forei_burn:INTEGER,milit_burn:INTEGER,other_burn:INTEGER,prop_manag:STRING,fed_code:STRING,nfdrs_fm:STRING,person_fir:STRING,gender:STRING,age:FLOAT,activity_w:STRING,horiz_dis:INTEGER,type_row:STRING,elevation:STRING,pos_slope:STRING,aspect:STRING,flame_lgth:INTEGER,spread_rat:INTEGER,serialid:INTEGER,latitude_longitude_appended:STRING


Waiting on bqjob_r3b51ce502a9c16fd_0000016e6d91cb9f_1 ... (4s) Current status: DONE   


In [68]:
%%bigquery 
select count(*) from WildLand_Incidents_staging.WildLandIncidents_2010

,f0_
0,53165


### WildLandIncidents2011

In [69]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.WildLandIncidents_2011 \
"gs://hermosillobazan_datasets/WildLand_Incidents_2010-2013/WildLandIncidents2011.csv"\
\state:STRING,fdid:STRING,inc_date:INTEGER,inc_no:STRING,exp_no:INTEGER,version:STRING,latitude:FLOAT,longitude:FLOAT,township:FLOAT,north_sou:STRING,range:INTEGER,east_west:STRING,section:INTEGER,subsection:STRING,meridian:STRING,area_type:STRING,fire_cause:STRING,hum_fact1:STRING,hum_fact2:STRING,hum_fact3:STRING,hum_fact4:STRING,hum_fact5:STRING,hum_fact6:STRING,hum_fact7:STRING,hum_fact8:STRING,fact_ign1:STRING,fact_ign2:STRING,supp_fact1:STRING,supp_fact2:STRING,supp_fact3:,heat_sourc:STRING,mob_prop:STRING,eq_inv_ign:STRING,nfdrs_id:STRING,weath_type:STRING,wind_dir:STRING,wind_speed:INTEGER,air_temp:INTEGER,rel_humid:INTEGER,fuel_moist:INTEGER,dangr_rate:STRING,bldg_inv:INTEGER,bldg_thr:INTEGER,acres_burn:FLOAT,crop_burn1:STRING,crop_burn2:STRING,crop_burn3:STRING,undet_burn:INTEGER,tax_burn:INTEGER,notax_burn:INTEGER,local_burn:INTEGER,couty_burn:INTEGER,st_burn:INTEGER,fed_burn:INTEGER,forei_burn:INTEGER,milit_burn:INTEGER,other_burn:INTEGER,prop_manag:STRING,fed_code:STRING,nfdrs_fm:STRING,person_fir:STRING,gender:STRING,age:FLOAT,activity_w:STRING,horiz_dis:INTEGER,type_row:STRING,elevation:STRING,pos_slope:STRING,aspect:STRING,flame_lgth:INTEGER,spread_rat:INTEGER,serialid:INTEGER,latitude_longitude_appended:STRING


Waiting on bqjob_r359b9f1def27a56_0000016e6d94bc36_1 ... (3s) Current status: DONE   


In [70]:
%%bigquery 
select count(*) from WildLand_Incidents_staging.WildLandIncidents_2011

,f0_
0,63462


## Create new dataset for modeled dataset2 titled "{source}_modeled"

### We are cutting 2012 and 2013 Since schemas change drastically

In [6]:
modeled_dataset_id = "WildLand_Incidents_modeled"
!bq --location=US mk --dataset {modeled_dataset_id}

Dataset 'trusty-wavelet-252622:WildLand_Incidents_modeled' successfully created.


## Create modeled tables

### Date Modeled Table - EDIT FROM STRING TO DATE IN BEAM -SEPERATE INTO YEAR, MONTH & DAY IN BEAM

In [50]:
%%bigquery 
CREATE TABLE WildLand_Incidents_modeled.Date AS 
SELECT * FROM
(SELECT DISTINCT inc_date AS date
 FROM WildLand_Incidents_staging.WildLandIncidents_2010
 UNION DISTINCT
 SELECT DISTINCT inc_date AS date
 FROM WildLand_Incidents_staging.WildLandIncidents_2011)
ORDER BY date

""


In [51]:
%%bigquery 
select count(*) from WildLand_Incidents_modeled.Date

,f0_
0,730


### Incident Modeled Tables

In [52]:
%%bigquery 
CREATE TABLE WildLand_Incidents_modeled.Incidents AS
SELECT * FROM
(SELECT serialid, state, fdid, inc_date AS date, inc_no, exp_no AS exposure_no, weath_type, wind_dir, wind_speed, air_temp, rel_humid, acres_burn, crop_burn1, crop_burn2, crop_burn3
 FROM WildLand_Incidents_staging.WildLandIncidents_2010)
UNION ALL
(SELECT serialid, state, fdid, inc_date AS date, inc_no, exp_no AS exposure_no, weath_type, wind_dir, wind_speed, air_temp, rel_humid, acres_burn, crop_burn1, crop_burn2, crop_burn3
 FROM WildLand_Incidents_staging.WildLandIncidents_2011)
ORDER BY serialid

""


In [30]:
%%bigquery 
CREATE TABLE WildLand_Incidents_modeled.Incident_2011 AS
SELECT * FROM

ORDER BY serialid

""


In [32]:
%%bigquery 
select count(*) from WildLand_Incidents_modeled.Incident_2011

,f0_
0,63462


### State Modeled Table

In [33]:
%%bigquery 
CREATE TABLE WildLand_Incidents_modeled.State AS
SELECT * FROM
(SELECT DISTINCT state 
 FROM WildLand_Incidents_staging.WildLandIncidents_2010
 UNION DISTINCT
 SELECT DISTINCT state
 FROM WildLand_Incidents_staging.WildLandIncidents_2011)
ORDER BY state

""


In [34]:
%%bigquery 
select count(*) from WildLand_Incidents_modeled.State

,f0_
0,52
